Advance Computer Vision - Project 1
Due May 12 by 11pm  Points 30  Submitting a website url Available Apr 26 at 8am - May 14 at 11pm 19 days
This assignment was locked May 14 at 11pm.
Dear Participant,

We welcome you all to the case-based project of this course. The case study (described below - 30 points) covers concepts taught in Part 1 (First 8 hours of ACV and transfer learning).

Question : FACE_DETECTION_Questions.ipynbView in a new window

PFB the link to the project brief:

ACV_Project1_FaceDetector_Project_Brief.pdfView in a new window

Link to the dataset & files:

https://drive.google.com/drive/folders/1C52IycU5w87CYq5kJDiZXEa0fXRlXr88?usp=sharing (Links to an external site.)Links to an external site.

 

Github Link for the submission:  https://classroom.github.com/a/WSWMHTtWLinks to an external site.

Regards

Program Office

#### In this problem we use "Transfer Learning" of an Object Detector model to detect any object according to the problem in hand.

Here, We are particularly interested in detecting faces in a given image.

#### To use the model first, we need to import the model and its supporting files for the model to function. 

We see the below steps to import the model.

In [46]:
!pip install numpy==1.16.1

### Import MobileNet model given in file `mn_model.py`

In [0]:
import os

In [48]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
### Import MobileNet model given in file `mn_model.py`

os.chdir('/content/drive/My Drive/R9 Advanced Computer Vision Project 1/Face Detection/')

from mn_model import mn_model

In [50]:
!ls

dataset				 mobilenet_1_0_224_tf.h5
_DS_Store			 output_test
face_generator.py		 predictor.py
face_generator.pyc		 ssd_box_encode_decode_utils.py
_gitattributes			 ssd_box_encode_decode_utils.pyc
_gitignore			 ssd_mobilenet_face_epoch_25_loss0.0916.h5
keras_layer_AnchorBoxes.py	 wider_extract.py
keras_layer_AnchorBoxes.pyc	 WIDER_train
keras_layer_L2Normalization.py	 wider_train_small.npy
keras_layer_L2Normalization.pyc  WIDER_train.zip
keras_ssd_loss.py		 WIDER_val
keras_ssd_loss.pyc		 wider_val_small.npy
mn_model.py			 WIDER_val.zip
mn_model.pyc


### Import the BatchGenerator and SSDLoss functions in given files `face_generator.py`, `keras_ssd_loss` and `ssd_box_encode_decode_utils.py` as well, used in MobileNet model

In [0]:
#### Import the BatchGenerator and SSDLoss functions as well, used in MobileNet model

from face_generator import BatchGenerator
from keras_ssd_loss import SSDLoss
from ssd_box_encode_decode_utils import SSDBoxEncoder, decode_y, decode_y2

In [0]:

import warnings
warnings.filterwarnings("ignore")

from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, LearningRateScheduler
from keras.callbacks import Callback
from keras import backend as K 
from keras.models import load_model
from math import ceil 
import numpy as np 
from termcolor import colored

from mn_model import mn_model
from face_generator import BatchGenerator
from keras_ssd_loss import SSDLoss
from ssd_box_encode_decode_utils import SSDBoxEncoder, decode_y, decode_y2

# training parameters
from keras import backend as K
import scipy.misc as sm


## Set the parameters for the model

#### We need to customize the model parameters according to our problem as given below.

#### Set n_classes (no.of classes) = 2, as we are interested in only face detection. 
#### `Face` will be one class and everything else comes under other class (we can call it as `background`)

#### Set class_names = ["background", "face"]

In [0]:
img_height =512
img_width = 512

img_channels = 3

n_classes =2 
class_names = ["background","face"]

scales = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # anchorboxes for coco dataset
aspect_ratios = [[0.5, 1.0, 2.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [0.5, 1.0, 2.0],
                 [0.5, 1.0, 2.0]] # The anchor box aspect ratios used in the original SSD300
two_boxes_for_ar1 = True
limit_boxes = True # Whether or not you want to limit the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are scaled as in the original implementation
coords = 'centroids' # Whether the box coordinates to be used as targets for the model should be in the 'centroids' or 'minmax' format, see documentation
normalize_coords = True

det_model_path = "./"

### Now, we have imported the model and its dependencies. The next thing is to import the dataset for the model to train on. For this, we are using the WIDER FACE dataset. 

#### To make the dataset available follow the steps given below.


1. Create a folder in your google drive for this project. 

2. Download the train and test dataset files given in .zip format into your drive folder you created for the project in step-1.

3. Set the project path variable according to the folders you created to use for this project in your google drive. 

      `project_path = "/content/drive/My Drive/DLCP/"`

4. Now, as we mount the drive the images will be available to use for training and testing but in zip format.

5. So, lets extract the images from the zipfiles by using the code given of zipfile module.







In [0]:
project_path = '/content/drive/My Drive/R9 Advanced Computer Vision Project 1/Face Detection/'

In [0]:
train_images_path = project_path + 'WIDER_train.zip'
test_images_path = project_path + 'WIDER_val.zip'

In [0]:
import zipfile
archive = zipfile.ZipFile(train_images_path, 'r')
archive.extractall()

In [0]:
archive = zipfile.ZipFile(test_images_path, 'r')
archive.extractall()

### Now, the images are available. The next thing we need is to get the labels for these images, so that we can use this information while training for detecting faces with the given model using transfer learning. 

#### Follow the below steps to get those labels available.


Load the  '' `wider_train_small.npy`'' file given to check the information given about the dataset. In this file you can see the information about each image in the dataset in a list with following elemets:
      

        1.   Image filename (str)
        2.   Image filename (str)
        3.   Image size (list) [height, width]
        4.   List of bounding box co-ordinates and Class label (list) [[a,b,c,d], Class label, ...]
        
        where,
        a,b,c,d are the four co-ordinates of the bounding box
        Class label is the position of object as mentioned in `class_names` list above.

In [0]:
import numpy as np

In [0]:
data = np.load('/content/drive/My Drive/R9 Advanced Computer Vision Project 1/Face Detection/wider_train_small.npy', allow_pickle=True).item()

In [0]:
data_test = np.load('/content/drive/My Drive/R9 Advanced Computer Vision Project 1/Face Detection/wider_val_small.npy', allow_pickle=True).item()

In [59]:
### Printed first element to check the above given information.

for key in data:
    print key
    print data[key]
    break

52--Photographers/52_Photographers_photographertakingphoto_52_582.jpg
['WIDER_train/images/52--Photographers/52_Photographers_photographertakingphoto_52_582.jpg', 'WIDER_train/images/52--Photographers/52_Photographers_photographertakingphoto_52_582.jpg', [300, 300], [[21, 667, 9, 655], 1]]


 As we can see from the above output all the information mentioned above is there for all the images.

### Now, load the files `wider_train.npy` and `wider_val.npy`

In [0]:
train_data = 'wider_train_small.npy'
test_data = 'wider_val_small.npy'

x = np.load(train_data)

In [0]:
train_data = 'wider_train_small.npy'
test_data = 'wider_val_small.npy'

x = np.load(train_data)
x_t = np.load(test_data)

### Now, call the imported model with the given parameters and freeze all the layers in the model with names not having ''`detection`'' word as prefix.

As we are not training the model from scratch, we are freezing all the above layers in the model having only last few layers while training to update their weights according to the problem in hand. This is called as **Transfer Learning**.

In [0]:
from keras import backend as K

In [63]:
# build the keras model
# this model is not retrained, we are doing it from scratch 

K.clear_session()
model, model_layer, img_input, predictor_sizes = mn_model(image_size=(img_height, img_width, img_channels), 
                                                                      n_classes = n_classes,
                                                                      min_scale = None, 
                                                                      max_scale = None, 
                                                                      scales = scales, 
                                                                      aspect_ratios_global = None, 
                                                                      aspect_ratios_per_layer = aspect_ratios, 
                                                                      two_boxes_for_ar1= two_boxes_for_ar1, 
                                                                      limit_boxes=limit_boxes, 
                                                                      variances= variances, 
                                                                      coords=coords, 
                                                                      normalize_coords=normalize_coords)

====> Model Specific data
====> Height, Width, Channels : 512 512 3


#### Write code to freeze all the layers in the above model with names not having ''`detection`'' word as prefix.

In [65]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in model.layers:
  if('detection' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

input_1
False
lambda1
False
lambda2
False
lambda3
False
conv1
False
conv1_bn
False
conv1_relu
False
conv_dw_1
False
conv_dw_1_bn
False
conv_dw_1_relu
False
conv_pw_1
False
conv_pw_1_bn
False
conv_pw_1_relu
False
conv_dw_2
False
conv_dw_2_bn
False
conv_dw_2_relu
False
conv_pw_2
False
conv_pw_2_bn
False
conv_pw_2_relu
False
conv_dw_3
False
conv_dw_3_bn
False
conv_dw_3_relu
False
conv_pw_3
False
conv_pw_3_bn
False
conv_pw_3_relu
False
conv_dw_4
False
conv_dw_4_bn
False
conv_dw_4_relu
False
conv_pw_4
False
conv_pw_4_bn
False
conv_pw_4_relu
False
conv_dw_5
False
conv_dw_5_bn
False
conv_dw_5_relu
False
conv_pw_5
False
conv_pw_5_bn
False
conv_pw_5_relu
False
conv_dw_6
False
conv_dw_6_bn
False
conv_dw_6_relu
False
conv_pw_6
False
conv_pw_6_bn
False
conv_pw_6_relu
False
conv_dw_7
False
conv_dw_7_bn
False
conv_dw_7_relu
False
conv_pw_7
False
conv_pw_7_bn
False
conv_pw_7_relu
False
conv_dw_8
False
conv_dw_8_bn
False
conv_dw_8_relu
False
conv_pw_8
False
conv_pw_8_bn
False
conv_pw_8_relu
False
conv

### After making the model ready for transfer learning, load the weights of the model given in file ''`mobilenet_1_0_224_tf.h5`''

In [0]:
#The pre-trained weights must exist in a folder in the current folder
model.load_weights('//content/drive/My Drive/R9 Advanced Computer Vision Project 1/Face Detection/mobilenet_1_0_224_tf.h5', by_name=True)

#### Using the functions given in the model, we are trying to divide the dataset into train and validation samples. Run the below code.

In [69]:
batch_size = 32
ssd_box_encoder = SSDBoxEncoder(img_height=img_height,
                                img_width=img_width,
                                n_classes=n_classes, 
                                predictor_sizes=predictor_sizes,
                                min_scale=None,
                                max_scale=None,
                                scales=scales,
                                aspect_ratios_global=None,
                                aspect_ratios_per_layer=aspect_ratios,
                                two_boxes_for_ar1=two_boxes_for_ar1,
                                limit_boxes=limit_boxes,
                                variances=variances,
                                pos_iou_threshold=0.5,
                                neg_iou_threshold=0.2,
                                coords=coords,
                                normalize_coords=normalize_coords)

train_dataset = BatchGenerator(images_path=train_data, 
                include_classes='all', 
                box_output_format = ['class_id', 'xmin', 'xmax', 'ymin', 'ymax'])

print ("==>TRAINING DATA")
print ("==> Parsing XML files ...")

train_dataset.parse_xml(
                  annotations_path=train_data,
                  image_set_path='None',
                  image_set='None',
                  classes = class_names, 
                  exclude_truncated=False,
                  exclude_difficult=False,
                  ret=False, 
                  debug = False)
print("==>Parsing XML Finished.")

print ("==>Generate training batches...")
train_generator = train_dataset.generate(
                 batch_size=batch_size,
                 train=True,
                 ssd_box_encoder=ssd_box_encoder,
                 equalize=True,
                 brightness=(0.5,2,0.5),
                 flip=0.5,
                 translate=((0, 20), (0, 30), 0.5),
                 scale=(0.75, 1.2, 0.5),
                 crop=False,
                 #random_crop = (img_height,img_width,1,3), 
                 random_crop=False,
                 resize=(img_height, img_width),
                 #resize=False,
                 gray=False,
                 limit_boxes=True,
                 include_thresh=0.4,
                 diagnostics=False)

print ("==>Training batch generation complete")

n_train_samples = train_dataset.get_n_samples()

print ("==>Total number of training samples = {}".format(n_train_samples))

# Now repeat above steps for validation data 

print ("==>VALIDATION")

val_dataset = BatchGenerator(images_path=test_data, include_classes='all', 
                box_output_format = ['class_id', 'xmin', 'xmax', 'ymin', 'ymax'])

print ("==> Parsing XML files ...")


val_dataset.parse_xml(
                  annotations_path=test_data,
                  image_set_path='None',
                  image_set='None',
                  classes = class_names, 
                  exclude_truncated=False,
                  exclude_difficult=False,
                  ret=False, 
                  debug = False)


print("==>Parsing XML Finished.")


print ("==>Generate training batches...")
val_generator = val_dataset.generate(
                 batch_size=batch_size,
                 train=True,
                 ssd_box_encoder=ssd_box_encoder,
                 equalize=False,
                 brightness=False,
                 flip=False,
                 translate=False,
                 scale=False,
                 crop=False,
                 #random_crop = (img_height,img_width,1,3), 
                 random_crop=False, 
                 resize=(img_height, img_width), 
                 #resize=False, 
                 gray=False,
                 limit_boxes=True,
                 include_thresh=0.4,
                 diagnostics=False)


print ("==>Training batch generation complete")

n_val_samples = val_dataset.get_n_samples()

print ("==>Total number of validation samples = {}".format(n_val_samples))

==>TRAINING DATA
==> Parsing XML files ...
==>Parsing XML Finished.
==>Generate training batches...
==>Training batch generation complete
==>Total number of training samples = 128
==>VALIDATION
==> Parsing XML files ...
==>Parsing XML Finished.
==>Generate training batches...
==>Training batch generation complete
==>Total number of validation samples = 60


### Now, lets setup things for training by initilaizing required variables like learning rate, epochs, optimizer and loss function(SSDLoss) to compile the model

In [0]:
# setting up training 

# batch_size and no.of epochs

batch_size = 16
num_epochs = 5

#Learning rate
base_lr = 0.002

# Optimizer
adam = Adam(lr=base_lr, beta_1=0.9, beta_2=0.999, epsilon=1e-6, decay = 0.0)

# Loss
ssd_loss = SSDLoss(neg_pos_ratio=2, n_neg_min=0, alpha=1.0, beta = 1.0)

# Compile
model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

### Lets add early stopping and model checkpoint layers on validation loss with some patience values and using fit_generator function to train the model on data generated batch-by-batch by a Python generator, `train_generator` object as generator.


We are using checkpoint to save the best model based on validation accuracy.

#### Write code for early_stopping and model_checkpoint layers. Using model.fit_generator train the model and save the best weight file.

In [74]:

filepath = '//content/drive/My Drive/R9 Advanced Computer Vision Project 1/Face Detection/mobilenet_1_0_224_tf.h5'

checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True,
                             save_weights_only=False, mode="min", period=1) # Checkpoint best validation model

stop = EarlyStopping(monitor="ssd_loss", patience=7, mode="min") # Stop early, if the validation error deteriorates

#reduce_lr = ReduceLROnPlateau(monitor="ssd_loss", factor=0.2, patience=10, min_lr=1e-7, verbose=1, mode="max")

callback_list = [checkpoint, stop]

model.fit_generator(train_generator, nb_epoch=num_epochs, steps_per_epoch=10, callbacks = callback_list)

Epoch 1/5
10/10 [==============================] - 211s 21s/step - loss: 0.3420
Epoch 2/5
10/10 [==============================] - 186s 19s/step - loss: 0.2776
Epoch 3/5
10/10 [==============================] - 187s 19s/step - loss: 0.2512
Epoch 4/5
10/10 [==============================] - 187s 19s/step - loss: 0.2344
Epoch 5/5
10/10 [==============================] - 187s 19s/step - loss: 0.2277


### Load the best saved model from above step and check predictions for test data using test_generator object to generate batches.

#### Write code in the below cell to load best saved model in the above step.

In [0]:
model_path = '/content/drive/My Drive/R9 Advanced Computer Vision Project 1/Face Detection/'

model_name = 'ssd_mobilenet_face_epoch_25_loss0.0916.h5'

model.load_weights(model_path + model_name,  by_name= True)

### Use the below function to plot the boundingbox in the test image to show the predictions.

In [0]:
def save_bb(path, filename, results, prediction=True):
  
  # print filename

  img = image.load_img(filename, target_size=(img_height, img_width))
  img = image.img_to_array(img)

  filename = filename.split("/")[-1]

  if(not prediction):
    filename = filename[:-4] + "_gt" + ".jpg"

  #fig,currentAxis = plt.subplots(1)
  currentAxis = plt.gca()

 # Get detections with confidence higher than 0.6.
  colors = plt.cm.hsv(np.linspace(0, 1, 25)).tolist()
  color_code = min(len(results), 16)
  print (colored("total number of bbs: %d" % len(results), "yellow"))
  for result in results:
    # Parse the outputs.

    if(prediction):
      det_label = result[0]
      det_conf = result[1]
      det_xmin = result[2]
      det_xmax = result[3]
      det_ymin = result[4]
      det_ymax = result[5]
    else :
      det_label = result[0]
      det_xmin = result[1]
      det_xmax = result[2]
      det_ymin = result[3]
      det_ymax = result[4]

    xmin = int(det_xmin)
    ymin = int(det_ymin)
    xmax = int(det_xmax)
    ymax = int(det_ymax)

    if(prediction):
      score = det_conf
    
    plt.imshow(img / 255.)
    
    label = int(int(det_label))
    label_name = class_names[label]
    # print label_name 
    # print label

    if(prediction):
      display_txt = '{:0.2f}'.format(score)
    else:
      display_txt = '{}'.format(label_name)

      
    # print (xmin, ymin, ymin, ymax)
    coords = (xmin, ymin), (xmax-xmin), (ymax-ymin)
    color_code = color_code-1 
    color = colors[color_code]
    currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
    currentAxis.text(xmin, ymin, display_txt, bbox={'facecolor':color, 'alpha':0.2})

  # y
  currentAxis.axes.get_yaxis().set_visible(False)
  # x
  currentAxis.axes.get_xaxis().set_visible(False)
  plt.savefig(path + filename, bbox_inches='tight')

  print ('saved' , path + filename)

  plt.clf()


#### Run the below code to create a folder with name output_test and get the predictions for the test images using model.predict()

In [79]:
!mkdir output_test

mkdir: cannot create directory ‘output_test’: File exists


In [80]:
from keras.preprocessing import image
from matplotlib import pyplot as plt

test_size = 16
test_generator = val_dataset.generate(
                 batch_size=test_size,
                 train=False,
                 ssd_box_encoder=ssd_box_encoder,
                 equalize=False,
                 brightness=False,
                 flip=False,
                 translate=False,
                 scale=False,
                 crop=False,
                 #random_crop = (img_height,img_width,1,3), 
                 random_crop=False, 
                 resize=(img_height, img_width), 
                 #resize=False,
                 gray=False,
                 limit_boxes=True,
                 include_thresh=0.4,
                 diagnostics=False)

print (colored("done.", "green"))

print (colored("now predicting...", "yellow"))

_CONF = 0.60 
_IOU = 0.15

for i in range(test_size):
    X, y, filenames = next(test_generator)

    y_pred = model.predict(X)
    y_pred_decoded = decode_y2(y_pred,
                             confidence_thresh=_CONF,
                            iou_threshold=_IOU,
                            top_k='all',
                            input_coords=coords,
                            normalize_coords=normalize_coords,
                            img_height=img_height,
                            img_width=img_width)


    np.set_printoptions(suppress=True)

    save_bb("./output_test/", filenames[i], y_pred_decoded[i])
    save_bb("./output_test/", filenames[i], y[i], prediction=False)


done.
now predicting...
total number of bbs: 3
('saved', './output_test/2_Demonstration_Demonstration_Or_Protest_2_367.jpg')
total number of bbs: 9
('saved', './output_test/2_Demonstration_Demonstration_Or_Protest_2_367_gt.jpg')
total number of bbs: 1
('saved', './output_test/30_Surgeons_Surgeons_30_533.jpg')
total number of bbs: 1
('saved', './output_test/30_Surgeons_Surgeons_30_533_gt.jpg')
total number of bbs: 3
('saved', './output_test/28_Sports_Fan_Sports_Fan_28_590.jpg')
total number of bbs: 9
('saved', './output_test/28_Sports_Fan_Sports_Fan_28_590_gt.jpg')
total number of bbs: 1
('saved', './output_test/18_Concerts_Concerts_18_102.jpg')
total number of bbs: 1
('saved', './output_test/18_Concerts_Concerts_18_102_gt.jpg')
total number of bbs: 3
('saved', './output_test/30_Surgeons_Surgeons_30_554.jpg')
total number of bbs: 6
('saved', './output_test/30_Surgeons_Surgeons_30_554_gt.jpg')
total number of bbs: 0
('saved', './output_test/25_Soldier_Patrol_Soldier_Patrol_25_467.jpg')
t

IndexError: ignored

<Figure size 432x288 with 0 Axes>

In the above step all the test images along with predictions are stored in output_test folder in this notebook environment. You can check the folder in Files section of the menu to left-side of screen in colab.

Each test image is used for predictions and is stored as 2 files.

one file is the original ground truth with <filename_gt.jpg>
second file is the prediction of the model on the image. with <filename.jpg>

### Visualize a test image to check predictions


#### Write code to show images: Using cv2.imshow() or matplotlib show any 3 test images and their predictions.

In [81]:
from keras.preprocessing import image
from matplotlib import pyplot as plt

test_size = 3
test_generator = val_dataset.generate(
                 batch_size=test_size,
                 train=False,
                 ssd_box_encoder=ssd_box_encoder,
                 equalize=False,
                 brightness=False,
                 flip=False,
                 translate=False,
                 scale=False,
                 crop=False,
                 #random_crop = (img_height,img_width,1,3), 
                 random_crop=False, 
                 resize=(img_height, img_width), 
                 #resize=False,
                 gray=False,
                 limit_boxes=True,
                 include_thresh=0.4,
                 diagnostics=False)

print (colored("done.", "green"))

print (colored("now predicting...", "yellow"))

_CONF = 0.60 
_IOU = 0.15

for i in range(test_size):
  X, y, filenames = next(test_generator)

  y_pred = model.predict(X)


  y_pred_decoded = decode_y2(y_pred,
                             confidence_thresh=_CONF,
                            iou_threshold=_IOU,
                            top_k='all',
                            input_coords=coords,
                            normalize_coords=normalize_coords,
                            img_height=img_height,
                            img_width=img_width)


  np.set_printoptions(suppress=True)

  save_bb("./output_test/", filenames[i], y_pred_decoded[i])
  save_bb("./output_test/", filenames[i], y[i], prediction=False)

done.
now predicting...
total number of bbs: 0
('saved', './output_test/25_Soldier_Patrol_Soldier_Patrol_25_467.jpg')
total number of bbs: 1
('saved', './output_test/25_Soldier_Patrol_Soldier_Patrol_25_467_gt.jpg')
total number of bbs: 1
('saved', './output_test/18_Concerts_Concerts_18_102.jpg')
total number of bbs: 1
('saved', './output_test/18_Concerts_Concerts_18_102_gt.jpg')
total number of bbs: 1
('saved', './output_test/49_Greeting_peoplegreeting_49_589.jpg')
total number of bbs: 1
('saved', './output_test/49_Greeting_peoplegreeting_49_589_gt.jpg')


<Figure size 432x288 with 0 Axes>